In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

__Creating functions to clean tweets__

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#import lang_detect
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from langdetect import detect

def stem_and_lemmatize(words):
    stems = stem_words(words)
    return " ".join(stems)


def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def only_english(mystr):
    if lang_detect(mystr)=='en':   #give up
        return mystr
    else :
        return ''


def clean_string(mystr):
    mystr=mystr.lower()
    mystr=re.sub(r"\\\w+", " ", mystr)
    mystr=re.sub(r"\@\w+"," ",mystr)
    mystr=re.sub(r"\#\w+"," ",mystr)
    mystr=re.sub(r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"," ",mystr)
    
    mystr=mystr[2:-1]
    
    cleantext = "".join([x.lower() if (x.isalpha() or x.isspace()) else ' ' for x in mystr])
    return cleantext

def remove_stop_words(mystr):
    word_list= mystr.split()
    cleaned_word_list=[w for w in word_list if w not in stop_words and len(w)>=2]
    
    return cleaned_word_list
   
    
def join_sep(mylist):
    return " ".join(mylist)
   


In [ ]:
# impcolumns=list(vertical_stack.columns[np.argsort(chi2score[0])[::-1]][:1000])  #from a previous run, now i have cut those cells
# #print(impcolumns)

# with open("impcolumns.txt", 'w') as f:
#     for s in impcolumns:
#         f.write(s + '\n')

#read these columns from a text file calledimpcolumns.txt if needed

with open('impcolumns.txt', 'r') as f:
    impcolumns = f.readlines()
    

    
impcolumns = [line[:-1] for line in impcolumns]
#print(impcolumns)
vertical_stack=impcolumns

import json

with open('approach2_FORGRAPH1.json') as f:
    data1 = json.load(f)

with open('approach2_FORGRAPH2.json') as f:
    data2 = json.load(f)
    
with open('approach4_final_data.json') as f:
    data3 = json.load(f)

__Loading data and packages__

In [ ]:
#f= approach_FORGRAPH_tweets_data.json'
import json
with open('approach4_final_data.json', 'r') as fp:
    tweet_data = json.load(fp)

In [ ]:
#Using oversampled data from drive

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

__Loading Classifier and prediction__

In [ ]:
import pickle
from sklearn.externals import joblib

In [ ]:
Pkl_Filename='DT_adaboost.pkl'
with open(Pkl_Filename, 'rb') as file:  
    classifier = clf = joblib.load(file)


classifier

In [ ]:
# X_new.to_csv('X_train_oversampled.csv', index = False, header=True)
# y_new.to_csv('y_train_oversampled.csv', index = False, header=True)
# X_test.to_csv('X_test.csv', index = False, header=True)
# y_test.to_csv('y_test.csv', index = False, header=True)

import collections
from collections import OrderedDict
from operator import itemgetter 

### Making tfidf from tweets and score


In [ ]:


def return_score(user_tweets):     #here user_tweets would be a dict with key as the username and value as the tweets 

     
    dataframes = {}
    dataframes_empty = {}


    for x in user_tweets:
    #print(x)
        
    
        try:
            file=pd.DataFrame(user_tweets[x],columns=['id','text'])
    
            file['clean_text']=file['text'].apply(clean_string)
            file['word_list']=file['clean_text'].apply(remove_stop_words)
            file['word_list']=file['word_list'].apply(stem_and_lemmatize)
            #file['word_list']=file['word_list'].apply(join_sep)
            #file['word_list']=file['word_list'].apply(only_english)
    
            file=file.dropna()
            file=file.loc[file['word_list']!='']
            
            ##############
    
    
            vectorizer = TfidfVectorizer()
        
            tfidfmatrix = vectorizer.fit_transform(file['word_list'])
            vocab = vectorizer.get_feature_names()

            tfidf_data=tfidfmatrix.toarray()
        
            tfidf_pd=pd.DataFrame(data=tfidf_data,columns=vocab,index=file['id'])
        
            tfidf_pd = tfidf_pd.drop(tfidf_pd.columns.difference(impcolumns),1)
       
            dataframes[x] = tfidf_pd
            #print("Success",x)
    
        except ValueError as e: #happens when the vocabulary is empty
        
            #print("Failure",x,file['text'])
            dataframes_empty[x]='N/A'
        
        
            continue
            
    tesla_followers=dataframes
    feature_list=impcolumns
    #UNIFORM PADDING
    for key in tesla_followers:
        test=tesla_followers[key]
    
        #print(len(test.columns))
    

        toadd=list(set(feature_list)-set(test.columns))


        toadd_df=pd.DataFrame(np.zeros((test.shape[0], len(toadd))),columns=toadd)
        test.reset_index(drop=True, inplace=True)
        toadd_df.reset_index(drop=True, inplace=True)


        test=pd.concat([test,toadd_df],axis=1)
    
        tesla_followers[key]=test
        #print(test.shape)
    
    #SCORE    
    tesla_tweet_prediction=[]
    score={}
    for key in tesla_followers:
        test=tesla_followers[key]
        current_user_pred=classifier.predict(test)
        tesla_tweet_prediction.append(current_user_pred)
        score[key]=sum(current_user_pred)/len(current_user_pred)   
    
    
    sorted_score = OrderedDict(sorted(score.items(), key=itemgetter(1),reverse=True))
    sorted_score.update(dataframes_empty)    #adding users with No tweets
    
    return sorted_score

In [ ]:
first_level={}
first_no_of_tweets={}

ctr=0

for k, v in data1['tesla'].items():
    #print(k)
    #print(v.keys())
    first_level[k]=v['tweets']
    first_no_of_tweets[k]=len(v['tweets'])
    
for k, v in data2['tesla'].items():
    #print(k)
    #print(v.keys())
    first_level[k]=v['tweets']
    first_no_of_tweets[k]=len(v['tweets'])
    
for k, v in data3['tesla'].items():
    #print(k)
    #print(v.keys())
    first_level[k]=v['tweets']
    first_no_of_tweets[k]=len(v['tweets'])
    
    
print(len(first_no_of_tweets))

In [ ]:

#user_name_neg = ['AbbyChe88615215', 'QzI2jJcBFlZmomK', 'sonjahahn', 'Melectroner', 'barcemo2', 'Noahgreenberg12', 'AngelLadyVenus', 'Sagar59781359', 'timarajae', 'Moaz80334541', 'ppmax17', 'LOVE_Raelian', 'SchieblerXavier', 'nazwanazwanazwa', 'DevrajBelgaonk1', 'MatyalAzher', 'kingsugenius', 'greyeaglespouch', 'max_cardaci', 'klmgorov', 'rohitpandya31', 'ctominay', 'imsmruti', 'Bogdan40542498', 'Zeeshan01344758', 'AdamarisVelez', '1uEkvO6odBsDImj', 'MariannaGardar1', 'MelAlex97543906', 'valtomsgv', 'jackrow94480700', 'Domi9915', 'LGszHBzn6H9Aoec', 'QunNguy81644285', 'dblank64', 'MuhammadGhozy9', 'adedivision', 'Frank91806042', 'Seb57466842', 'HYnYang1999', 'KhengKouy', 'Saqlainsharif14', 'Raphael79724082', 'WisdomOgbonna12', 'karthikadr1', 'ibrahemka93', 'artemisia_annie', 'njtktm', 'BJ_Timson', 'MhHoussem', 'Owenchoi9', 'IdrisRockingidd', 'eHo9gixZa47kwcW', 'VickyGo13', 'IdreesAshrafMir', 'P4NT2', 'ICaldicott', 'gabi_emeli', 'LoRaine_fr', 'MarwaanWilliam1', 'StanWoodman4', 'FPrefekt', 'DevPate71188540', '7aDOxFkBKvyfl6', '__Nadeeka', 'priyankamoorthy', 'VivekPa63657629', 'dicksuck4money', 'shai_trakz', 'badal_khosla', 'SwapnilDurgeka1', 'barry_member', 'tsaharsharaf', 'SudipSi66921834', 'ColinCa42350619', 'JoeChen56203494', 'HapuNipuna', 'Whatsup65692095', 'mestevper4mance', 'AasimMushtaq10', 'CamilloGasparri', 'ZachFre06174485', 'Kien12092440', 'VRocchis', 'Charlie_WWright', 'Mathewsmutale9', 'Joshau72077726', 'benbetterb', 'indiana11011100', 'Pavan28843768', 'Andreaarman1', 'CoombesF', 'Soumy21134508', 'nadinelivgmail1', 'ChaudharyAvin', 'LoApa15', 'rachit_mangal', 'SooryanRajeev', 'AbhilashO8', 'aatrees', 'Muham83137969', 'RouelAndrew', 'ModiboDilika', 'LindaSouv', 'TommyTurkia', 'Griee4', 't_elevs', 'immeet6']

In [ ]:
#print(len(user_name_neg))

In [ ]:
print(len(first_level))
#c=0
first_level_list= return_score(first_level)
# for x in first_level:
#     if x in user_name_neg:
#         c+=1
# print(c)
        

In [ ]:
len(nodes_score)

In [ ]:
#node_classification using text
second_level={}
for k, v in data1['tesla'].items():
    second_level[k] = {}
    follwer_list = []
    user_tweets={}
    if isinstance(v, dict):
        for i, j in v.items():
            if not (i == "tweets" or i == 'location'):
                    follwer_list.append(i)
                    if j['tweets']!="error":
                        user_tweets[i] = j['tweets']
    second_level[k] = return_score(user_tweets)
    
for k, v in data2['tesla'].items():
    second_level[k] = {}
    follwer_list = []
    user_tweets={}
    if isinstance(v, dict):
        for i, j in v.items():
            if not (i == "tweets" or i == 'location'):
                    follwer_list.append(i)
                    if j['tweets']!="error":
                        user_tweets[i] = j['tweets']
    second_level[k] = return_score(user_tweets)
                        
for k, v in data3['tesla'].items():
    second_level[k] = {}
    follwer_list = []
    user_tweets={}
    if isinstance(v, dict):
        for i, j in v.items():
            if not (i == "tweets" or i == 'location'):
                    follwer_list.append(i)
                    if j['tweets']!="error":
                        user_tweets[i] = j['tweets']
    second_level[k] = return_score(user_tweets)               

In [ ]:
nodes_score = {}

for i in list(second_level.values()):
    for (j,k) in i.items():
            nodes_score[j] = k

In [ ]:
for (j,k) in return_score(first_level).items():
        nodes_score[j] = k

nodes_score['tesla'] = 1

In [ ]:
#recording edges
nodes = []
edges=[]
count = 0
nodes.append('tesla')
for i,j in data1['tesla'].items():
    nodes.append(i)
    tup_l1 = ("tesla",i)
    edges.append(tup_l1)
    count+= 1
    if len(j.keys()) > 0:
        a = list(j.keys())
        a.remove('tweets')
        a.remove('location')
        for k in a:
                count+= 1
                tup = (i,k)
                nodes.append(k)
                edges.append(tup)

for i,j in data2['tesla'].items():
    nodes.append(i)
    tup_l1 = ("tesla",i)
    edges.append(tup_l1)
    count+= 1
    if len(j.keys()) > 0:
        a = list(j.keys())
        a.remove('tweets')
        a.remove('location')
        for k in a:
                count+= 1
                tup = (i,k)
                nodes.append(k)
                edges.append(tup)

for i,j in data3['tesla'].items():
    nodes.append(i)
    tup_l1 = ("tesla",i)
    edges.append(tup_l1)
    count+= 1
    if len(j.keys()) > 0:
        a = list(j.keys())
        a.remove('tweets')
        a.remove('location')
        for k in a:
                count+= 1
                tup = (i,k)
                nodes.append(k)
                edges.append(tup)
print(count)   

In [ ]:
#split in train and test
#node_based sampling
from sklearn.model_selection import train_test_split
unlabeled_nodes = [i for i in nodes_score if nodes_score[i] == 'N/A']

for k in nodes:
    if not(k in nodes_score.keys()):
        unlabeled_nodes.append(k)

nodes_data = list(nodes_score.keys())
for i in unlabeled_nodes:
    try:
        nodes_data.remove(i)
    except:
        continue
train_nodes, test_nodes = train_test_split(nodes_data)

In [ ]:
threshold = 0.5

In [ ]:
import copy
nodes_score_copy = copy.deepcopy(nodes_score)

for j in test_nodes:
    unlabeled_nodes.append(j)

for i in unlabeled_nodes:
    nodes_score_copy[i] = threshold

for k in range(50):
    delta_change = 0
    for l in unlabeled_nodes:
        neigbours = []
        for n,m in edges:
            if n == l:
                neigbours.append(m)
            if m == l:
                neigbours.append(n)
        neigbour_scores = [nodes_score_copy[a] for a in neigbours]
        old_val = nodes_score_copy[l]
        nodes_score_copy[l] = np.mean(neigbour_scores)
        #delta_change += abs(nodes_score_copy[l] - old_val)

In [ ]:
predicted_values = [1 if nodes_score_copy[i] > threshold else 0 for i in test_nodes]
actual_values = [1 if nodes_score[i] > threshold else 0 for i in test_nodes]

from sklearn.metrics import f1_score
f1_score(actual_values, predicted_values, average='binary')

In [ ]:
[0.2,0.3,0.4,0.5]
[0.9585832619251643,0.851159793814433, 0.6146179401993355, 0.23239436619718312]

import json
with open('hierarchy_scores.json', 'w') as fp:
    json.dump(second_level, fp)

In [ ]:
#HOMOPHILY
one_one = 0
zero_one = 0
zero_zero = 0
invalid = 0
for i,j in edges:
    if (i in nodes_score.keys()) and (j in nodes_score.keys()):
        if nodes_score[i] >= 0.5 and nodes_score[j] >= 0.5:
            one_one += 1
        elif nodes_score[i] < 0.5 and nodes_score[j] >= 0.5:
            zero_one += 1
        elif nodes_score[i] >= 0.5 and nodes_score[j] < 0.5:
            zero_one += 1
        else:
            zero_zero += 1
    else:
        invalid += 1

        


In [ ]:
print(one_one)
print(zero_one)
print(zero_zero)
print(invalid)

print(one_one+zero_one+zero_zero+invalid)

print(count)

In [ ]:
edges_population = []

for _ in range(one_one+zero_zero):
    edges_population.append(0)
for _ in range(zero_one):
    edges_population.append(1)
    

In [ ]:
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats

In [ ]:
pip install bootstrapped

In [ ]:
samples = np.random.choice(edges_population, 1000, replace = True)

In [ ]:
bs.bootstrap(samples, stat_func=bs_stats.mean)

In [ ]:
zero_one/(one_one+zero_zero)